# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

<ipython-input-1-1499ab5789c7>:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

#from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# original library
sys.path.append('/home/hiroki/research/dcase2021_task2/src/functions')
import common as com
import pytorch_modeler as modeler
from pytorch_model import Conditional_VAE as Model
#from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
log_file = config['IO_OPTION']['OUTPUT_ROOT']+'/train_{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_file, '00_train.py')

## Setting

In [5]:
# Setting seed
modeler.set_seed(42)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
#dev_dir = INPUT_ROOT + "/dev_data"
#add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = ['ToyCar', 'gearbox', 'valve', 'ToyTrain', 'pump', 'fan', 'slider']
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
OUT_SCORE_DIR = OUTPUT_ROOT + '/score'
OUT_PRED_DIR = OUTPUT_ROOT + '/pred'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
os.makedirs(OUT_SCORE_DIR, exist_ok=True)
os.makedirs(OUT_PRED_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/HDD1TB/research/dcase2021_task2/output/ID_Conditional_VAE/baseline/config.yaml'

In [7]:
torch.cuda.is_available()

True

In [8]:
#ext_data = pd.read_pickle(f'{INPUT_ROOT}/{machine_types[0]}_features.pkl')
#dataloaders_dict = modeler.make_dataloader(ext_data)

In [9]:
#for sample in dataloaders_dict['train']:
#    hoge = sample['features']

## training

In [10]:
#############################################################################
# run
#############################################################################
def run(machine_type):
    com.tic()
    
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dataloader
    ext_data = pd.read_pickle(f'{INPUT_ROOT}/{machine_type}_features.pkl')
    dataloaders_dict = modeler.make_dataloader(ext_data)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    # out path
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    score_out_path = OUT_SCORE_DIR + '/{}_score.csv'.format(machine_type)
    pred_out_path = OUT_PRED_DIR + '/{}_pred.csv'.format(machine_type)
    logger.info('TRAINING')
    # parameter setting
    in_features = ext_data['train']['features'].shape[1]
    mid_size = config['param']['mid_size']
    latent_size = config['param']['latent_size']
    net = Model(in_features, mid_size, latent_size)
    optimizer = optim.Adam(net.parameters(), lr=1e-3)
    num_epochs = config['param']['num_epochs']
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e-2, 
                                              max_lr=1e-3, epochs=num_epochs, steps_per_epoch=len(dataloaders_dict['train']))
    # training
    output_dicts = modeler.train_net(net, dataloaders_dict, optimizer, scheduler, num_epochs, writer, model_out_path, score_out_path, pred_out_path)
    
    com.toc()

In [11]:
machine_types

['ToyCar', 'gearbox', 'valve', 'ToyTrain', 'pump', 'fan', 'slider']

In [12]:
for machine_type in machine_types:
    run(machine_type)

2021-04-20 22:26:00,082 - 00_train.py - INFO - TARGET MACHINE_TYPE: ToyCar
2021-04-20 22:26:00,083 - 00_train.py - INFO - MAKE DATA_LOADER
2021-04-20 22:26:00,117 - 00_train.py - INFO - TRAINING


use: cuda:0


2021-04-20 22:26:01,929 - pytorch_modeler.py - INFO - train
100%|██████████| 24/24 [00:01<00:00, 21.81it/s]
2021-04-20 22:26:03,032 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 3/3 [00:00<00:00, 23.75it/s]
2021-04-20 22:26:03,181 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 3/3 [00:00<00:00, 24.73it/s]


,AUC,pAUC
Source_0,0.530900,0.503947
Source_1,0.207550,0.492632
Source_2,0.950300,0.743684
Target_0,0.517650,0.509474
Target_1,0.212300,0.479474
Target_2,0.923000,0.612105
mean,0.556950,0.556886
h_mean,0.387608,0.543427


2021-04-20 22:26:04,161 - pytorch_modeler.py - INFO - epoch:1/100, train_losses:23971.553294, val_AUC_hmean:0.387608, val_pAUC_hmean:0.543427, best_flag:True
2021-04-20 22:26:04,162 - pytorch_modeler.py - INFO - train
100%|██████████| 24/24 [00:00<00:00, 42.57it/s]
2021-04-20 22:26:04,729 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 3/3 [00:00<00:00, 24.13it/s]
2021-04-20 22:26:04,874 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 3/3 [00:00<00:00, 24.96it/s]


...

100%|██████████| 3/3 [00:00<00:00, 18.60it/s]
2021-04-20 22:36:01,413 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 3/3 [00:00<00:00, 18.03it/s]
2021-04-20 22:36:01,602 - pytorch_modeler.py - INFO - epoch:55/100, train_losses:6.233767, val_AUC_hmean:0.472455, val_pAUC_hmean:0.555877, best_flag:False
2021-04-20 22:36:01,603 - pytorch_modeler.py - INFO - train
100%|██████████| 24/24 [00:00<00:00, 42.50it/s]
2021-04-20 22:36:02,170 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 3/3 [00:00<00:00, 19.96it/s]
2021-04-20 22:36:02,338 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 3/3 [00:00<00:00, 19.67it/s]
2021-04-20 22:36:02,512 - pytorch_modeler.py - INFO - epoch:56/100, train_losses:3.572306, val_AUC_hmean:0.472145, val_pAUC_hmean:0.556197, best_flag:False
2021-04-20 22:36:02,513 - pytorch_modeler.py - INFO - train
100%|██████████| 24/24 [00:00<00:00, 43.82it/s]
2021-04-20 22:36:03,063 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 3/3

100%|██████████| 24/24 [00:00<00:00, 43.63it/s]
2021-04-20 22:36:16,654 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 3/3 [00:00<00:00, 21.02it/s]
2021-04-20 22:36:16,816 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 3/3 [00:00<00:00, 21.63it/s]
2021-04-20 22:36:16,981 - pytorch_modeler.py - INFO - epoch:72/100, train_losses:0.480186, val_AUC_hmean:0.472446, val_pAUC_hmean:0.556198, best_flag:False
2021-04-20 22:36:16,982 - pytorch_modeler.py - INFO - train
100%|██████████| 24/24 [00:00<00:00, 43.18it/s]
2021-04-20 22:36:17,540 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 3/3 [00:00<00:00, 19.43it/s]
2021-04-20 22:36:17,712 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 3/3 [00:00<00:00, 21.02it/s]
2021-04-20 22:36:17,877 - pytorch_modeler.py - INFO - epoch:73/100, train_losses:0.426777, val_AUC_hmean:0.472279, val_pAUC_hmean:0.556121, best_flag:False
2021-04-20 22:36:17,878 - pytorch_modeler.py - INFO - train
100%|██████████| 24/

2021-04-20 22:36:31,449 - pytorch_modeler.py - INFO - train
100%|██████████| 24/24 [00:00<00:00, 43.57it/s]
2021-04-20 22:36:32,002 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 3/3 [00:00<00:00, 20.49it/s]
2021-04-20 22:36:32,165 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 3/3 [00:00<00:00, 19.96it/s]
2021-04-20 22:36:32,335 - pytorch_modeler.py - INFO - epoch:89/100, train_losses:0.334046, val_AUC_hmean:0.472077, val_pAUC_hmean:0.556177, best_flag:False
2021-04-20 22:36:32,336 - pytorch_modeler.py - INFO - train
100%|██████████| 24/24 [00:00<00:00, 43.62it/s]
2021-04-20 22:36:32,888 - pytorch_modeler.py - INFO - valid_source
100%|██████████| 3/3 [00:00<00:00, 19.38it/s]
2021-04-20 22:36:33,061 - pytorch_modeler.py - INFO - valid_target
100%|██████████| 3/3 [00:00<00:00, 21.18it/s]
2021-04-20 22:36:33,226 - pytorch_modeler.py - INFO - epoch:90/100, train_losses:0.338696, val_AUC_hmean:0.472922, val_pAUC_hmean:0.559219, best_flag:False
2021-04-20 22:36:33,

,AUC,pAUC
Source_0,0.546550,0.484662
Source_1,0.866300,0.787105
Source_2,0.953832,0.757013
Target_0,0.558450,0.506654
Target_1,0.827245,0.784017
Target_2,0.947664,0.731356
mean,0.783340,0.675134
h_mean,0.741920,0.647159


elapsed time: 97.088124752 [sec]
